In [2]:
import pandas as pd

In [3]:
file = 'raw/city-county-prop-tax-analysis.xlsx'
dtype = {
    'County #': str
}
# Note: read_excel() pulls from Excel files like read_csv() pulls from .csv's
raw_total_tax = pd.read_excel(file, dtype=dtype, sheet_name='TaxesByCo')
raw_total_res_tax = pd.read_excel(file, dtype=dtype, sheet_name='Class4ResTaxesByCo')
population = pd.read_excel(file, dtype=dtype, sheet_name='County Population')

inflation = pd.read_excel('raw/bls-cpi-history.xlsx')

In [4]:
inflation.head()

,year,month,rel_cpi
0,2002,June,1.00
1,2003,June,1.02
2,2004,June,1.05
3,2005,June,1.08
4,2006,June,1.13


In [11]:
df = pd.concat([population.sum(), raw_total_res_tax[:56].sum(), raw_total_tax[:56].sum()], axis=1)
df.columns = ['population','residential_tax_collections','total_tax_collections']
df = df[2:]

In [12]:
df['total_per_capita'] = df['total_tax_collections'] / df['population']
df['res_per_capita'] = df['residential_tax_collections'] / df['population']
df = df.reset_index().rename(columns={'index':'year'})
df = df.merge(inflation[['year','rel_cpi']], on='year')

In [17]:
base_total = df.iloc[0]['total_per_capita']
df['total_inflationary_increase'] = base_total * df['rel_cpi']

In [18]:
df.head()

,year,population,residential_tax_collections,total_tax_collections,total_per_capita,res_per_capita,rel_cpi,total_inflationary_increase
0,2002,911667,3.60068e+08,8.03576e+08,881.436,394.955,1.00,881.435810
1,2003,919630,3.84529e+08,8.46035e+08,919.973,418.134,1.02,899.064526
2,2004,930009,4.20479e+08,9.05352e+08,973.488,452.123,1.05,925.507600
3,2005,940102,4.50839e+08,9.68321e+08,1030.02,479.564,1.08,951.950675
4,2006,952692,4.76675e+08,1.01949e+09,1070.12,500.345,1.13,996.022465


In [19]:
df.to_csv('data/state-tax-trend.csv', index=False)